In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from config import api_key
import gmaps.datasets
import gmaps
import time

# Drunk Driving Fatal Collisions vs Bars Location

In [ ]:
filepath_TPS_alcohol = "Resources/Fatal_Collisions.csv"
TPS_alcohol_df = pd.read_csv(filepath_TPS_alcohol)
TPS_alcohol_df

In [ ]:
lats=[]
lngs=[]
for index,row in TPS_alcohol_df.iterrows():
    if row["ALCOHOL"] == "Yes": 
        lats.append(row[0])
        lngs.append(row[1])

In [ ]:
drunk_drivers = {"longitudes":lngs, "latitudes":lats}
drunk_drivers_df = pd.DataFrame(drunk_drivers)
drunk_drivers_df.head()

In [ ]:
target_coordinates = "43.6532 , -79.3832"
target_search = ("Bar","Pub")
target_radius = 140000
#the radius of toronto is 21 kms
target_type = ("Bar","Pub")
result_list=[]
# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": api_key
    }

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response=requests.get(base_url, params=params)


In [ ]:
flag = True
count=0
while flag:
    count+=1
    print("Working on Page ",count)

    #response=requests.get(base_url, params=params)
    places_data = response.json()
    lat=[]
    lng=[]
    results=places_data['results']
    Geo=pd.DataFrame(results)['geometry']
    name=pd.DataFrame(results)['name']
    for x in Geo:
        location=pd.DataFrame(x)['location']
        lat.append(location['lat'])
        lng.append(location['lng'])
    

    latitude=pd.DataFrame(lat)
    longitude=pd.DataFrame(lng)
    barname=pd.DataFrame(name)
    barname.join(latitude)
    barname['C'] = barname.reset_index().index
    latitude['C'] = latitude.reset_index().index
    longitude['C'] = longitude.reset_index().index
#barname.join(latitude.set_index('C'))
#longitude.join(latitude.set_index('C'))
    sample2=pd.merge(barname,latitude,on='C')
    sample1=pd.merge(sample2,longitude,on='C')
    result_list.append(sample1)

    if('next_page_token' in places_data):
        paramaters = {
        "key": api_key,
        "pagetoken":places_data['next_page_token']
        }
        time.sleep(2)
       # print(paramaters)
        #print(base_url)
        response=requests.get(base_url, paramaters)
    else:
        flag=False

#response = requests.get(base_url, params=params)
result_df = pd.concat(result_list)[['name','0_x','0_y']]

#print(result_df)
result_df2=pd.DataFrame(result_df)
result_df2

In [ ]:
gmaps.configure(api_key=api_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(drunk_drivers_df,max_intensity=1)
fig.add_layer(heat_layer)
marker_layer = gmaps.symbol_layer(result_df2[["0_x","0_y"]],stroke_opacity=1,fill_opacity=1)
fig.add_layer(marker_layer)
fig

# Fatal Collisions vs Neighborhood Mean Income

In [ ]:
income_df=pd.read_csv('Population Income Neighborhood.csv')
income_df.head()

In [ ]:
#fig_income = gmaps.figure()

#heat_layer2 = gmaps.heatmap_layer(income_df[["Lat","Long"]],max_intensity=3)
#fig_income.add_layer(heat_layer2)
#income_integer = income_df["Neighborhood Mean Income"].astype(int)
#heat_layer2.point_radius = 3
#marker_layer = gmaps.symbol_layer(TPS_df[["Y","X"]],stroke_opacity=0.2,fill_opacity=0.2,scale=income_df["Neighborhood Mean Income"])
#fig_income.add_layer(marker_layer)
#fig_income

# Fatal Collisions vs Neighborhood Income

In [ ]:
filepath_TPS = "Accident_info (1).csv"
TPS_df = pd.read_csv(filepath_TPS)

TPS_df

In [ ]:
lats1=[]
lngs1=[]
for index,row in TPS_df.iterrows():
    lats1.append(row[1])
    lngs1.append(row[2])

In [ ]:
popinc_df=pd.read_csv('Population Income Neighborhood.csv')
popinc_df.head()

In [ ]:
# a=[]
# for z in range(0,len(popinc_df['Neighborhood Mean Income'])):
#     a.append((popinc_df.iloc[z,3]/10000).astype(int))
    
# x=(np.array(a)*2).astype(int)
# len(x)
# b=x.tolist()

In [ ]:
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(TPS_df[["LATITUDE","LONGITUDE"]],max_intensity=1)

# fig.add_layer(heat_layer)

# marker_layer = gmaps.symbol_layer(popinc_df[["Lat","Long"]],stroke_opacity=0.4,fill_opacity=0.6,scale=b)# scale is income parameter
# fig.add_layer(marker_layer)
# fig

In [ ]:
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(lat1, lon1, lat2, lon2):
   R = 6373.0
   lat1 = radians(lat1)
   lon1 = radians(lon1)
   lat2 = radians(lat2)
   lon2 = radians(lon2)
   dlon = lon2 - lon1
   dlat = lat2 - lat1
   a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
   c = 2 * atan2(sqrt(a), sqrt(1 - a))
   return R * c
print(calculate_distance(52.2296756,21.0122287,52.406374,16.9251681))

In [ ]:
popinc_df["Number of Accidents"]=0
popinc_df.head()

In [ ]:
popinc_df=popinc_df.drop(axis='columns',columns=["Unnamed: 0"])

In [ ]:
popinc_df=popinc_df.set_index('Category')

In [ ]:
popinc_df.head()

In [ ]:
for w in range(len(TPS_df)):
    mini=100000
    for y in range(len(popinc_df)):
           
        dist = calculate_distance(TPS_df.iloc[w,1],TPS_df.iloc[w,2],popinc_df.iloc[y,2],popinc_df.iloc[y,3])
        if (dist<mini):
            mini=dist
            y1=y
    popinc_df.iloc[y1,4]=popinc_df.iloc[y1,4]+1
popinc_df.head()

In [ ]:
plt.scatter(popinc_df['Neighborhood Mean Income'],popinc_df['Number of Accidents'],marker='o',facecolors="green",edgecolors="brown")
plt.xlabel('Neighborhood Mean Income($)')
plt.ylabel('Number of Accidents')
plt.title('Accidents vs Neighborhood Income')
plt.savefig('Accidents vs Neighborhood Income')

# Type of Vehicle involved in Fatal Collisions

In [ ]:
filepath_TPS_V = "Fatal_Collisions.csv"
TPS_V_df = pd.read_csv(filepath_TPS_V)
a=TPS_V_df[TPS_V_df['IMPACTYPE']=='Pedestrian Collisions']
a

In [ ]:
lats_v=[]
lngs_v=[]
for index,row in TPS_V_df.iterrows():
   lats_v.append(row[0])
   lngs_v.append(row[1])
    
fig_v = gmaps.figure()


symbol_layer=gmaps.symbol_layer(TPS_V_df[["Y","X"]],stroke_opacity=0.1,fill_opacity=0.1)
fig_v.add_layer(symbol_layer)
fig_v

# Cause of Collision

In [ ]:
filepath_coll = "Fatal_Collisions.csv"
fatal_coll=pd.read_csv(filepath_coll)
fatal_coll.head()

In [ ]:
fatal_col1=fatal_coll[['LATITUDE','LONGITUDE','STREET1','STREET2','DATE','TIME','YEAR','ROAD_CLASS','District','LOCCOORD','TRAFFCTL','VISIBILITY','LIGHT',
                       'INVTYPE','INVAGE','VEHTYPE','MANOEUVER','DRIVACT','DRIVCOND','PEDESTRIAN','CYCLIST','AUTOMOBILE','MOTORCYCLE','TRUCK','SPEEDING','REDLIGHT','ALCOHOL','Neighbourh','ObjectId']]

In [ ]:
fatal_col1.columns

In [ ]:
fatal_col2=fatal_col1.rename(columns={"LOCCOORD":"INTERSECTION","ROAD_CLASS":"ACCIDENT LANE","INVTYPE":"VICTIM","INVAGE":"VICTIM AGE"})
fatal_col2.columns

In [ ]:
fatal_col3=fatal_col2[['LATITUDE','LONGITUDE','DATE','TIME','YEAR','ACCIDENT LANE','District','INTERSECTION','VISIBILITY','LIGHT',
                       'VICTIM','VICTIM AGE','VEHTYPE','DRIVACT','PEDESTRIAN','CYCLIST','AUTOMOBILE','MOTORCYCLE','TRUCK','SPEEDING','REDLIGHT','ALCOHOL','Neighbourh','ObjectId']]
fatal_col3.columns

In [ ]:
fatal_col4=fatal_col3
fatal_col4.columns

In [ ]:
def cause_finder(r):
    
    if ((r['ALCOHOL'] == 'Yes') and (r['SPEEDING'] == 'Yes')):
        v = "Alcohol & Speeding"
    elif (r['SPEEDING'] == 'Yes'):
        v = "Speeding"
    elif (r['ALCOHOL'] == 'Yes'):
        v = "Alcohol"
    elif ((r['DRIVACT']=='Disobeyed Traffic Control') or (r['DRIVACT']=='Exceeding Speed Limit') or 
          (r['DRIVACT']=='Failed to Yield Right of Way') or (r['DRIVACT']=='Following too Close') or 
         (r['DRIVACT']=='Improper Lane Change') or (r['DRIVACT']=='Improper Passing') or
         (r['DRIVACT']=='Improper Turn') or (r['DRIVACT']=='Lost control') or (r['DRIVACT']=='Speed too Fast For Condition') or
         (r['DRIVACT']=='Wrong Way on One Way Road') or (r['DRIVACT']=='Speed too Slow')):
        v= r['DRIVACT']
    elif ((r['VISIBILITY']=='Rain') or (r['VISIBILITY']=='Snow') or (r['VISIBILITY']=='Freezing Rain') or
          (r['VISIBILITY']=='Fog, Mist, Smoke, Dust')):
        v= r['VISIBILITY']
    else:
        v='Unknown'
 
    return v

fatal_col4['Cause'] = fatal_col4.apply(cause_finder, axis = 1)
#fatal_col4.loc[fatal_col4['Cause_Real']=='']
fatal_col4['Cause'].value_counts()

In [ ]:
fatal_col4.head()

In [ ]:
Accident_info=fatal_col4[['LATITUDE', 'LONGITUDE', 'DATE', 'TIME', 'YEAR', 'ACCIDENT LANE',
       'District', 'INTERSECTION','VICTIM',
       'VICTIM AGE', 'VEHTYPE', 'DRIVACT', 'PEDESTRIAN', 'CYCLIST',
       'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'Cause','Neighbourh']]
Accident_info.to_csv("Accident_info.csv")
fatal_col4.to_csv("Fatal Collisions.csv")
Accident_info.head()

In [ ]:
Cause_df=Accident_info.groupby('Cause').count()
Cause=Cause_df.drop(axis='index',index='Unknown')

In [ ]:
plt.barh(Cause.index,width=Cause['DRIVACT'],height=0.8,color="green")
plt.xlabel('Number of Accidents')
plt.ylabel('Cause')
plt.title('Accidents vs Cause')
plt.savefig('Accidents vs Cause')